Below you can find the code to import data from niania.pl website

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

how many pages nianiapl has for chosen location:

In [2]:
pages = 8

create set to store offer links:

In [3]:
links = set()

create dictionary from which we will create df later:

In [4]:
dict = {'date': [], 'address': [], 'salary': [], 'employment': [], 'author': [], 'experience': [],
           'nannys_age': [], 'additional_requirements': [], 'language1': [], 'language2':[], 'children': [],
           'number_children': []}

getting all links and adding offer links:

In [5]:
for page in range(1, pages+1):
    page_link = f'''https://niania.pl/praca/warszawa?page={page}'''
    response = requests.get(page_link)
    soup = BeautifulSoup(response.text, 'html.parser')

    tags = soup('a')

    for tag in tags:
        offer_link = tag.get('href',None)  
        if type(offer_link) == str and offer_link.startswith('https://niania.pl/szukam/'):
            links.add(offer_link)

how many offers do we have:

In [6]:
print(f'number of offers: ', len(links))

number of offers:  235


iterate through each offer to create connection with every page and get necessary data:

In [7]:
for link in links:

    # create connection with every page:

    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup_str = str(soup)

    # get necessary data:

    lista = soup.find('ul', class_="list-style-none summary")
    lista_oczekiwania = soup.find_all('li', class_="pico-tick-before")

    languages = soup.find_all('li', class_='pb-3 icon-lang-polish-before')

    try: #first languge
        language1 = languages[0].get_text().strip()
    except IndexError:
        language1 = None

    try: #second languge
        language2 = languages[1].get_text().strip()
    except IndexError:
        language2 = None

    try: #nannys preferred age
        nannys_age = lista_oczekiwania[3].find("strong").get_text()
    except (IndexError, AttributeError):
        nannys_age = None


    try: #announcement author
        author = lista.select("li:nth-of-type(7)")[0].find("strong").get_text()
    except AttributeError:
        author = None

    try: #nannys exeprience
        experience = lista_oczekiwania[2].find("strong").get_text()
    except (IndexError, AttributeError):
        experience = None

    #date
    date = soup.find('time').get_text()

    #address
    address = lista.select("li:nth-of-type(5)")[0].find("span").get_text()

    #salary
    salary = lista.select("li:nth-of-type(4)")[0].find("span").get_text()

    # type of job
    employment = lista.select("li:nth-of-type(3)")[0].find_all("span")
    e = [employment.get_text() for employment in employment]
    employment_list = [e.strip() for e in e]

    additional_requirements = []

    # extra requirements
    for element in range(4, len(lista_oczekiwania)):
        requirement = lista_oczekiwania[element].get_text().splitlines()
        d = ' '.join(requirement)
        d = d.strip()
        additional_requirements.append(d)

    # number of children
    child = soup.find_all('li', class_='pb-2')

    number_children = len(child)
    children = []

    try:
        if len(child) > 1:
            for element in range(0, len(child)):
                p = child[element-1].get_text()
                pp = p.strip()
                children.append(pp)
        else:
            dd = child[0].get_text()
            ddd = dd.strip()
            children.append(ddd)
    except IndexError:
            children.append(None)

    # data gathering:

    headers = list(dict.keys())

    dane = [date, address, salary, employment_list, author, experience, nannys_age, additional_requirements,
            language1, language2, children, number_children]

    for value in range(0, len(dane)):
        try:
            dict[headers[value]].append(dane[value])
        except Exception:
            dict[headers[value]].append(None)

creating df with all data:

In [9]:
df = pd.DataFrame(dict)

Creating a DataFrame to display the results

In [14]:
#checking the number of null values
null_counts = df.isnull().sum()

# checking the number of non-null values 
non_null_counts = df.notnull().sum()

result_df = pd.DataFrame({
    'Column': df.columns,
    'Null Values': null_counts,
    'Non-Null Values': non_null_counts
})
result_df.iloc[:, 1:].sort_values('Null Values', ascending = False)

,Null Values,Non-Null Values
language2,197,38
author,62,173
language1,59,176
experience,38,197
nannys_age,22,213
date,0,235
address,0,235
salary,0,235
employment,0,235
additional_requirements,0,235


creating csv from df:

In [11]:
df.to_csv('niania_pl_final.csv')